## Tuning starts here

 ### VAE DR + CLF

In [8]:
import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
tf.config.experimental.list_physical_devices('XLA_GPU')
# tf.config.experimental_list_devices()

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:2', device_type='XLA_GPU')]

In [3]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
tf.test.is_gpu_available()

False

In [2]:
###Variational Autoencoder to get the latent layer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback
from keras.losses import mse, binary_crossentropy
import keras

import pydot
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score

from multiprocessing import Process, Manager

Using TensorFlow backend.


In [3]:
def set_Data(data):
    ppmi = pd.read_csv('../../datasets/preprocessed/trans_processed_PPMI_data.csv')
    ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
    ppmi.set_index('Sentrix_position', inplace=True)
    ppmi = ppmi.transpose()

    encoder = LabelEncoder()
    label = encoder.fit_transform(ppmi['Category'])

    tr = ppmi.drop(['Category'], axis=1)
    X = tr.values
    y = label
    print(X.shape)
    print(y.shape)

    print("StratifiedSampling check")
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    split.get_n_splits(X, y)

    for train_index, test_index in split.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, data['y_test'] = y[train_index], y[test_index]

    print("Oversampling check")
    oversampler = SMOTE(random_state=42)
    X_train_sampled, data['y_train_sampled'] = oversampler.fit_resample(X_train, y_train)
    print("Scaling check")
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_sampled)
    data['X_train_scaled_1'] = X_train_scaled[:247].reshape((1, -1))
    data['X_train_scaled_2'] = X_train_scaled[247:].reshape((1, -1))
    data['X_test_scaled'] = scaler.transform(X_test)
    
    print("Returning check")

manager = Manager()
data = manager.dict()

print("CHECKPOINT1")
#     p = Process(target=set_Data, args=(X_train_scaled, X_test_scaled, y_train_sampled, y_test,))
p = Process(target=set_Data, args=(data,))
print("CHECKPOINT2")
p.start()
print("CHECKPOINT3")
p.join()

y_train = data['y_train_sampled']
y_test = data['y_test']
X_train = np.append(data['X_train_scaled_1'], data['X_train_scaled_2']).reshape(494, 747668)
X_test = data['X_test_scaled']

CHECKPOINT1
CHECKPOINT2
CHECKPOINT3


/home/rz296/miniconda3/envs/partII/lib/python3.6/multiprocessing/popen_fork.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,25

(436, 747668)
(436,)
StratifiedSampling check
Oversampling check
Scaling check
Returning check


In [4]:
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [5]:
#Define custom variational layer
class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

In [6]:
# Initialise variables and Set hyperparameters
original_dim = X_train.shape[1] #747668
latent_dim = 100

batch_size = 50 # controls the number of training samples to 
                # work through before the model's internal parameters are updated
epochs = 50

In [7]:
# VAE model = encoder + decoder

############ENCODER##################
inputs = Input(shape=(original_dim, ), name='encoder_input')
#mean and log_var are the vectors of size `latent_dim`
z_mean_linear = Dense(latent_dim, kernel_initializer='glorot_uniform', name='z_mean')(inputs)
z_mean_batchnorm = BatchNormalization()(z_mean_linear)
z_mean_encoded = Activation('relu')(z_mean_batchnorm)

z_log_var_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(inputs)
z_log_var_batchnorm = BatchNormalization()(z_log_var_linear)
z_log_var_encoded = Activation('relu')(z_log_var_batchnorm)

# return the encoded and randomly sampled z vector
# Takes two keras layers as input to the custom sampling function layer with a `latent_dim` output
# note that "output_shape" isn't necessary with the TensorFlow backend

#### actually not very sure how and why use sampling??#####
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean_encoded, z_log_var_encoded])

# Model to compress input
encoder = Model(rnaseq_input, z_mean_encoded)


############DECODER##################
decoder_to_reconstruct = Dense(original_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
reconstructed_output = decoder_to_reconstruct(z)

########### instantiate VAE model##########
adam = optimizers.Adam(lr=learning_rate)
vae_layer = CustomVariationalLayer()([inputs, reconstructed_output])
vae = Model(inputs, vae_layer)
vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

vae.summary()


    
#Train the model
vae.fit(X_train, y_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, X_test))

score = vae.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'rnaseq_input' is not defined

### NN


### MLP

In [11]:
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

import talos
from talos.utils.gpu_utils import multi_gpu

import os

Using TensorFlow backend.


In [54]:
###Hyperparameters for MLP
p = {'lr': (0.8, 1.0, 3),
     'first_neuron':[256, 512],#,32, 64, 128
     'hidden_layer_neuron':[32], #64, 128, 256
     'batch_size': [10], #, 20, 30
     'epochs': [10], #, 20, 30
     'dropout': [0],#this means try every .1 value between 0 and .5
     'kernel_initializer': ['uniform'], #,'normal'
     'optimizer': ['Adam'], #'Nadam'
     'losses': ['binary_crossentropy'],
     'activation':['relu'], #, 'elu', 'tanh'
     'last_activation': ['sigmoid'] #, 'softmax'
    }

In [55]:
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="2";  
def mlp_model(x_train, y_train, x_val, y_val, params):
    # Build the model.
    # Anyhow give parameters first
    mlp = Sequential([
      Dense(params['first_neuron'], activation=params['activation'], input_shape=(747668,), kernel_initializer=params['kernel_initializer']),
      Dropout(params['dropout'], seed=42),
      Dense(params['hidden_layer_neuron'], activation=params['activation']),
      Dropout(params['dropout'], seed=42),
    #   Dense(8, activation='relu'),
    #   Dropout(Dropout(0.50, seed=42)),
      Dense(2, activation=params['last_activation']),
    ])

    # split a single job to multiple GPUs
#     mlp = multi_gpu(mlp)

    # Compile the model
    mlp.compile(
      optimizer=params['optimizer'],
      loss=params['losses'],
      metrics=['accuracy'],
    )

    # Train the data
    history = mlp.fit(
      x_train, # training data
      to_categorical(y_train), # training targets
      epochs=params['epochs'],
      batch_size=params['batch_size'],
    )

    return history, mlp

In [56]:
#Tune hyperparameters using talos
## To start fast, limit the permutation to 1/100 of the original permutation
scan_object = talos.Scan(x=X_train,
                         y=y_train, 
                         params=p,
                         model=mlp_model,
                         experiment_name='mlp')



  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/10
243/243 [==============================] - 22s 91ms/step - loss: 99.2364 - accuracy: 0.5720
Epoch 2/10
243/243 [==============================] - 22s 91ms/step - loss: 50.4819 - accuracy: 0.7737
Epoch 3/10
243/243 [==============================] - 22s 90ms/step - loss: 26.3427 - accuracy: 0.8148
Epoch 4/10
243/243 [==============================] - 22s 90ms/step - loss: 6.0931 - accuracy: 0.9300
Epoch 5/10
243/243 [==============================] - 22s 91ms/step - loss: 6.6742 - accuracy: 0.9465
Epoch 6/10
243/243 [==============================] - 22s 90ms/step - loss: 5.2511 - accuracy: 0.9486
Epoch 7/10
243/243 [==============================] - 22s 91ms/step - loss: 9.7458 - accuracy: 0.9342
Epoch 8/10
243/243 [==============================] - 22s 91ms/step - loss: 2.9741 - accuracy: 0.9527
Epoch 9/10
243/243 [==============================] - 22s 90ms/step - loss: 6.6655 - accuracy: 0.9465
Epoch 10/10
243/243 [==============================] - 22s 91ms/step - loss: 2.



 17%|█▋        | 1/6 [03:45<18:46, 225.38s/it]

Epoch 1/10
243/243 [==============================] - 22s 90ms/step - loss: 162.5748 - accuracy: 0.5597
Epoch 2/10
243/243 [==============================] - 22s 90ms/step - loss: 83.9551 - accuracy: 0.7922
Epoch 3/10
243/243 [==============================] - 22s 89ms/step - loss: 40.1386 - accuracy: 0.8683
Epoch 4/10
243/243 [==============================] - 22s 89ms/step - loss: 23.6784 - accuracy: 0.9321
Epoch 5/10
243/243 [==============================] - 22s 91ms/step - loss: 31.4243 - accuracy: 0.9403
Epoch 6/10
243/243 [==============================] - 22s 91ms/step - loss: 4.7200 - accuracy: 0.9588
Epoch 7/10
243/243 [==============================] - 22s 91ms/step - loss: 11.4621 - accuracy: 0.9506
Epoch 8/10
243/243 [==============================] - 22s 91ms/step - loss: 2.9521 - accuracy: 0.9712
Epoch 9/10
243/243 [==============================] - 22s 89ms/step - loss: 6.0054 - accuracy: 0.9630
Epoch 10/10
243/243 [==============================] - 22s 89ms/step - loss



 33%|███▎      | 2/6 [07:29<15:00, 225.00s/it]

...



 67%|██████▋   | 4/6 [18:41<09:43, 291.71s/it]

Epoch 1/10
243/243 [==============================] - 44s 182ms/step - loss: 171.5801 - accuracy: 0.5802
Epoch 2/10
243/243 [==============================] - 44s 181ms/step - loss: 45.4014 - accuracy: 0.7243
Epoch 3/10
243/243 [==============================] - 44s 181ms/step - loss: 9.8214 - accuracy: 0.7942
Epoch 4/10
243/243 [==============================] - 44s 180ms/step - loss: 8.6678 - accuracy: 0.8004
Epoch 5/10
243/243 [==============================] - 44s 181ms/step - loss: 2.4885 - accuracy: 0.8086
Epoch 6/10
243/243 [==============================] - 44s 181ms/step - loss: 2.0026 - accuracy: 0.7963
Epoch 7/10
243/243 [==============================] - 44s 181ms/step - loss: 1.3848 - accuracy: 0.8066
Epoch 8/10
243/243 [==============================] - 44s 183ms/step - loss: 0.3892 - accuracy: 0.8045
Epoch 9/10
243/243 [==============================] - 44s 179ms/step - loss: 0.3906 - accuracy: 0.8025
Epoch 10/10
243/243 [==============================] - 44s 181ms/step 



 83%|████████▎ | 5/6 [26:12<05:39, 339.51s/it]

Epoch 1/10
243/243 [==============================] - 43s 179ms/step - loss: 169.7645 - accuracy: 0.6111
Epoch 2/10
243/243 [==============================] - 44s 180ms/step - loss: 36.7431 - accuracy: 0.7922
Epoch 3/10
243/243 [==============================] - 44s 179ms/step - loss: 11.6468 - accuracy: 0.8683
Epoch 4/10
243/243 [==============================] - 44s 183ms/step - loss: 2.3361 - accuracy: 0.8807
Epoch 5/10
243/243 [==============================] - 44s 182ms/step - loss: 1.4793 - accuracy: 0.8992
Epoch 6/10
243/243 [==============================] - 44s 180ms/step - loss: 0.2897 - accuracy: 0.9198
Epoch 7/10
243/243 [==============================] - 44s 181ms/step - loss: 0.1192 - accuracy: 0.9239
Epoch 8/10
243/243 [==============================] - 45s 183ms/step - loss: 0.1588 - accuracy: 0.9239
Epoch 9/10
243/243 [==============================] - 44s 182ms/step - loss: 0.3978 - accuracy: 0.9177
Epoch 10/10
243/243 [==============================] - 45s 184ms/step



100%|██████████| 6/6 [34:00<00:00, 340.03s/it]


In [58]:
# accessing the results data frame
display(scan_object.data)

# accessing epoch entropy values for each round
display(scan_object.learning_entropy)

# access the summary details
scan_object.details

####Documentation for training: as the number of neuron increases, accuracy does not necessarily increase, however the time taken for training increases significantly

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
0,10,2.722449,0.958848,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
1,10,2.506606,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
2,10,0.333380,0.936214,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
3,10,0.121178,0.934156,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
4,10,0.389267,0.804527,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
5,10,1.754559,0.911523,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam


,loss,accuracy
0,1.612864,2.292603
1,1.627731,2.292070
2,1.081373,2.292943
3,1.102660,2.294472
4,0.954518,2.298395
5,0.808678,2.296348


experiment_name                     mlp
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
reduction_threshold                 0.2
reduction_metric                val_acc
complete_time            02/03/20/22:25
x_shape                   (348, 747668)
y_shape                          (348,)
dtype: object

In [59]:
sorted_so = scan_object.data.sort_values(by=["accuracy"], ascending=False)
sorted_so

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
1,10,2.506606,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam
0,10,2.722449,0.958848,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
2,10,0.333380,0.936214,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
3,10,0.121178,0.934156,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
5,10,1.754559,0.911523,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.933333,Adam
4,10,0.389267,0.804527,relu,10,0,10,512,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam


In [66]:
df1 = pd.DataFrame(sorted_so.iloc[:1])
# final_params_info = df1
df = pd.concat([df1, final_params_info]).sort_values(by=['accuracy'], ascending=False)
display(df)

final_params_info

,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
3,10,2.908766e-14,1.000000,relu,10,0,10,64,32,uniform,sigmoid,binary_crossentropy,0.800000,Adam
1,10,2.506606e+00,0.971193,relu,10,0,10,256,32,uniform,sigmoid,binary_crossentropy,0.866667,Adam


,round_epochs,loss,accuracy,activation,batch_size,dropout,epochs,first_neuron,hidden_layer_neuron,kernel_initializer,last_activation,losses,lr,optimizer
3,10,2.908766e-14,1.0,relu,10,0,10,64,32,uniform,sigmoid,binary_crossentropy,0.8,Adam


In [31]:
#Clear GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()

In [11]:
# Evaluate on Test set
# evaluate() returns an array containing the test loss followed by any metrics we specified. 
mlp.evaluate(
  X_test,
  to_categorical(y_test)
)

88/88 [==============================] - 0s 3ms/step


[100.73245377974077, 0.6477272510528564]

In [2]:
import gc
import ctypes

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.decomposition import FastICA

from sklearn.feature_selection import SelectFromModel

from multiprocessing import Process, Manager, RawArray, Array


def set_Data(data):
    ppmi = pd.read_csv('../../datasets/preprocessed/trans_processed_PPMI_data.csv')
    ppmi.rename(columns={'Unnamed: 0':'Sentrix_position'}, inplace=True)
    ppmi.set_index('Sentrix_position', inplace=True)
    ppmi = ppmi.transpose()

    encoder = LabelEncoder()
    label = encoder.fit_transform(ppmi['Category'])

    tr = ppmi.drop(['Category'], axis=1)
    X = tr.values
    y = label
    print(X.shape)
    print(y.shape)

    print("StratifiedSampling check")
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=29)
    split.get_n_splits(X, y)

    for train_index, test_index in split.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, data['y_test'] = y[train_index], y[test_index]

    print("Oversampling check")
    oversampler = SMOTE(random_state=42)
    X_train_sampled, data['y_train_sampled'] = oversampler.fit_resample(X_train, y_train)
    print("Scaling check")
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_sampled)
    print("Shape of X_train_sacled before splitting", X_train_scaled.shape)
    data['X_train_scaled_1'] = X_train_scaled[:247].reshape((1, -1))
    data['X_train_scaled_2'] = X_train_scaled[247:].reshape((1, -1))
    data['X_test_scaled'] = scaler.transform(X_test)
    
    print("Returning check")


Using TensorFlow backend.


In [3]:
manager = Manager()
data = manager.dict()

print("CHECKPOINT1")
#     p = Process(target=set_Data, args=(X_train_scaled, X_test_scaled, y_train_sampled, y_test,))
p = Process(target=set_Data, args=(data,))
print("CHECKPOINT2")
p.start()
print("CHECKPOINT3")
p.join()

y_train_sampled = data['y_train_sampled']
y_test = data['y_test']
X_train_scaled = np.append(data['X_train_scaled_1'], data['X_train_scaled_2']).reshape(432, 747668)
X_test_scaled = data['X_test_scaled']

CHECKPOINT1
CHECKPOINT2
CHECKPOINT3


/home/rz296/miniconda3/envs/partII/lib/python3.6/multiprocessing/popen_fork.py:73: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,25

(436, 747668)
(436,)
StratifiedSampling check
Oversampling check
Scaling check
Shape of X_train_sacled before splitting (432, 747668)
Returning check


In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
# print(y_train_sampled)
# print(X_train_scaled)
print ("Shape of final train and test sets:", X_train_scaled.shape, X_test_scaled.shape)
    
C_options = [0.001, 0.01, 0.1, 1, 100, 1000]
n_components = [20, 25, 30, 35, 40, 45, 50, 55, 60]

#-------------------------------------------------------
# ### PCA

# for n in n_components:
#     pca = PCA(n_components=n)
#     X_train = pca.fit_transform(X_train_scaled)
#     X_test = pca.transform(X_test_scaled)
#     print("PCA reduced check")
    
#     var = np.sum((pca.explained_variance_ratio_))
#     print("Total variance covered:", var)
#-------------------------------------------------------

### UMAP

for n in n_components:
    umap = UMAP(n_components=n)
    X_train = umap.fit_transform(X_train_scaled)
    X_test = umap.transform(X_test_scaled)

#-------------------------------------------------------

# ICA
# for n in n_components:
#     ica = FastICA(n_components=n)
#     X_train = ica.fit_transform(X_train_scaled)
#     X_test = ica.transform(X_test_scaled)

    param_grid = [
        {
            'C': C_options,
        },
    ]
    
    print("Shape of X train:", X_train.shape)
    print("Shape of y train:", y_train_sampled.shape)
    
    grid = GridSearchCV(LogisticRegression(max_iter=1000, penalty='l1', class_weight='balanced', tol=0.1, solver='saga'), 
                        param_grid=param_grid, 
                        scoring="precision", 
                        cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42), 
                        n_jobs=-1)
    grid.fit(X_train, y_train_sampled)
    # evaluation metric is f1 scoring
    print("Mean score of precision of the best C:", grid.best_score_)
    print("With UMAP=",n,"and l1, the best params are:")
    print(grid.best_params_, "for n_compo=", n)
    
    clf = grid.best_estimator_
    print(clf)
    y_pred = clf.predict(X_train)
    cm = confusion_matrix(y_train_sampled, y_pred)
    print(cm)

    y_test_pred = clf.predict(X_test)
    cm_test = confusion_matrix(y_test, y_test_pred)
    pre = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    acc = accuracy_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    print(cm_test)
    print("precision of testing set:", pre)
    print("recall of testing set", recall)
    print("accuracy of testing set", acc)
    print("f1 of testing set", f1)
    

## L1 Regularisation
# alpha=[0.001,0.01, 0.1]
# for a in alpha:
#     sel_ = SelectFromModel(Lasso(alpha=a))
#     sel_.fit(X_train_scaled, y_train_sampled)
#     X_train_selected = sel_.transform(X_train_scaled)
#     X_test_selected = sel_.transform(X_test_scaled)

#     param_grid = [
#         {
#             'C': C_options
#         },
#     ]

#     ####Use L1 LR for clf
#     grid = GridSearchCV(LogisticRegression(max_iter=500, penalty='l2', solver='saga'), param_grid=param_grid, scoring="accuracy", cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42), n_jobs=3)
#     grid.fit(X_train_selected, y_train)
#     mean_scores = np.array(grid.cv_results_['mean_test_score'])
#     print("FS using Regularisation with alpha=", a, "and l2:")
#     print(grid.cv_results_['params'])
#     print(mean_scores)
#     print(grid.best_params_)


# ###TEST for FS using different
# ####Lasso:
# sel_lasso = SelectFromModel(Lasso(alpha=0.01, tol=0.01))
# sel_lasso.fit(X_train_scaled, y_train)
# X_train_selected = sel_lasso.transform(X_train_scaled)
# X_test_selected = sel_lasso.transform(X_test_scaled)
# clf_lasso = LogisticRegression(max_iter=1000, penalty='l2', solver='saga', C=100).fit(X_train_selected, y_train)
# y_pred_lasso = clf_lasso.predict(X_test_selected)

# ####LogReg:
# sel_logreg = SelectFromModel(LogisticRegression(C=1000, penalty='l1', solver='saga', tol=0.01))
# sel_logreg.fit(X_train_scaled, y_train)
# X_train_selected = sel_logreg.transform(X_train_scaled)
# X_test_selected = sel_logreg.transform(X_test_scaled)
# clf_logreg = LogisticRegression(max_iter=1000, penalty='l2', solver='saga', C=0.1).fit(X_train_selected, y_train)
# y_pred_logreg = clf_logreg.predict(X_test_selected)

# print("Accuracy of LassoFS+LogReg:", accuracy_score(y_test, y_pred_lasso))
# print("Accuracy of LogRegFS+LogReg:", accuracy_score(y_test, y_pred_logreg))




Shape of final train and test sets: (432, 747668) (131, 747668)
Shape of X train: (432, 20)
Shape of y train: (432,)
Mean score of precision of the best C: 0.538347102863232
With UMAP= 20 and l1, the best params are:
{'C': 0.1} for n_compo= 20
LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.1, verbose=0,
                   warm_start=False)
[[114 102]
 [ 83 133]]
[[12 26]
 [34 59]]
precision of testing set: 0.6941176470588235
recall of testing set 0.6344086021505376
accuracy of testing set 0.5419847328244275
f1 of testing set 0.6629213483146066
Shape of X train: (432, 25)
Shape of y train: (432,)
Mean score of precision of the best C: 0.546622326551904
With UMAP= 25 and l1, the best params are:
{'C': 100} for n_compo= 25
LogisticRegression(C=100, class_weight=

In [36]:
y_test

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1])

In [37]:
y_train_sampled

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,